In [1]:
# Cell 1: Imports & Load Model-Ready Data

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

import joblib

# Load preprocessed features and target
X = pd.read_csv("c:\\Users\\Pc\\Desktop\\cardiac-risk-awareness\\notebooks\\data\\processed\\X_scaled.csv")
y = pd.read_csv("c:\\Users\\Pc\\Desktop\\cardiac-risk-awareness\\notebooks\\data\\processed\\y.csv").squeeze()

print("Features shape:", X.shape)
print("Target shape:", y.shape)
print("Target distribution:")
print(y.value_counts())

X.head()

Features shape: (5160, 44)
Target shape: (5160,)
Target distribution:
target
0.0    4007
1.0    1153
Name: count, dtype: int64


,BMI,BPMeds,age,ca,cigsPerDay,cp,currentSmoker,diaBP,diabetes,education,...,glucose_missing,heartRate_missing,oldpeak_missing,prevalentHyp_missing,prevalentStroke_missing,restecg_missing,slope_missing,sysBP_missing,thal_missing,totChol_missing
0,-0.088785,0,1.435909,0.0,0.0,typical angina,0,82.0,0,2.0,...,1,0,0,1,1,0,0,0,0,0
1,-0.088785,0,1.887489,3.0,0.0,asymptomatic,0,82.0,0,2.0,...,1,0,0,1,1,0,0,0,0,0
2,-0.088785,0,1.887489,2.0,0.0,asymptomatic,0,82.0,0,2.0,...,1,0,0,1,1,0,0,0,0,0
3,-0.088785,0,-1.499358,0.0,0.0,non-anginal,0,82.0,0,2.0,...,1,0,0,1,1,0,0,0,0,0
4,-0.088785,0,-1.047778,0.0,0.0,atypical angina,0,82.0,0,2.0,...,1,0,0,1,1,0,0,0,0,0


In [ ]:
# Cell 2: Encode Categorical Features

from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print("="*70)
print("CATEGORICAL ENCODING")
print("="*70)
print(f"Categorical columns found: {categorical_cols}")

if categorical_cols:
    # Label encode each categorical feature
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col])
        label_encoders[col] = le
        print(f"✓ Encoded {col}")
    print(f"\n✓ All categorical features encoded")
else:
    print("✓ No categorical features found")
    label_encoders = {}

print(f"Final shape: {X.shape}")
print(f"Data types:\n{X.dtypes.value_counts()}")
print("="*70)

In [11]:
# Cell 2: Split Features & Target (Train/Test)

# Stratified split (preserves target distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

print("="*70)
print("TRAIN/TEST SPLIT")
print("="*70)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

print(f"\nTarget distribution (train): {y_train.value_counts().to_dict()}")
print(f"Target distribution (test): {y_test.value_counts().to_dict()}")

assert X_train.shape[0] == y_train.shape[0], "Train size mismatch!"
assert X_test.shape[0] == y_test.shape[0], "Test size mismatch!"

print("\n✓ Stratified split validated")

# Cell 2: Encode Categorical Features

from sklearn.preprocessing import LabelEncoder

# Identify categorical columns
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

print(f"Categorical columns to encode: {categorical_cols}")

# Label encode each categorical feature
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le
    print(f"✓ Encoded {col}: {le.classes_.tolist()}")

print(f"\nAll features now numeric. Shape: {X.shape}")

TRAIN/TEST SPLIT
X_train shape: (4128, 44)
X_test shape: (1032, 44)
y_train shape: (4128,)
y_test shape: (1032,)

Target distribution (train): {0.0: 3206, 1.0: 922}
Target distribution (test): {0.0: 801, 1.0: 231}

✓ Stratified split validated
Categorical columns to encode: []

All features now numeric. Shape: (5160, 44)


In [12]:
# Cell 3: Train–Validation Split (Stratified)

# Stratified split (critical for medical datasets to preserve class balance)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("="*70)
print("TRAIN/VALIDATION SPLIT (STRATIFIED)")
print("="*70)
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

print(f"\nClass distribution (train): {y_train.value_counts().to_dict()}")
print(f"Class distribution (val): {y_val.value_counts().to_dict()}")

print("\n✓ Stratification ensures balanced medical dataset splits")

TRAIN/VALIDATION SPLIT (STRATIFIED)
X_train shape: (4128, 44)
X_val shape: (1032, 44)
y_train shape: (4128,)
y_val shape: (1032,)

Class distribution (train): {0.0: 3206, 1.0: 922}
Class distribution (val): {0.0: 801, 1.0: 231}

✓ Stratification ensures balanced medical dataset splits


In [13]:
# Cell 4: Train Baseline Logistic Regression

# Logistic Regression with balanced class weights (critical for medical data)
log_reg = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="liblinear",
    random_state=42
)

log_reg.fit(X_train, y_train)

print("="*70)
print("BASELINE LOGISTIC REGRESSION")
print("="*70)
print(f"✓ Model trained on {X_train.shape[0]} samples")
print(f"✓ Features: {X_train.shape[1]}")
print(f"✓ Class weights: balanced (handles imbalance)")
print(f"✓ Solver: liblinear (medical-grade stability)")

print(f"\nModel coefficients shape: {log_reg.coef_.shape}")
print(f"Intercept: {log_reg.intercept_[0]:.4f}")

BASELINE LOGISTIC REGRESSION
✓ Model trained on 4128 samples
✓ Features: 44
✓ Class weights: balanced (handles imbalance)
✓ Solver: liblinear (medical-grade stability)

Model coefficients shape: (1, 44)
Intercept: -1.4414


In [14]:
# Cell 5: Predictions & Probabilities

# Make predictions on validation set
y_val_pred = log_reg.predict(X_val)
y_val_proba = log_reg.predict_proba(X_val)[:, 1]

print("="*70)
print("VALIDATION PREDICTIONS")
print("="*70)
print(f"Prediction shape: {y_val_pred.shape}")
print(f"Probability shape: {y_val_proba.shape}")
print(f"Unique predictions: {np.unique(y_val_pred)}")
print(f"Probability range: [{y_val_proba.min():.4f}, {y_val_proba.max():.4f}]")

print(f"\nPrediction distribution:")
print(pd.Series(y_val_pred).value_counts().to_dict())

print("\n✓ Predictions & probabilities computed")

VALIDATION PREDICTIONS
Prediction shape: (1032,)
Probability shape: (1032,)
Unique predictions: [0. 1.]
Probability range: [0.0666, 0.9997]

Prediction distribution:
{0.0: 696, 1.0: 336}

✓ Predictions & probabilities computed


In [15]:
# Cell 6: Evaluation Metrics (Core Artifact)

# Calculate all metrics
accuracy = accuracy_score(y_val, y_val_pred)
precision = precision_score(y_val, y_val_pred, zero_division=0)
recall = recall_score(y_val, y_val_pred, zero_division=0)
f1 = f1_score(y_val, y_val_pred, zero_division=0)
roc_auc = roc_auc_score(y_val, y_val_proba)
cm = confusion_matrix(y_val, y_val_pred)

print("="*70)
print("VALIDATION METRICS (CARDIAC RISK FOCUS)")
print("="*70)
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}  ← CRITICAL (identifies high-risk patients)")
print(f"F1 Score:  {f1:.4f}")
print(f"ROC-AUC:   {roc_auc:.4f}")

print(f"\nConfusion Matrix:")
print(f"  True Negatives:  {cm[0, 0]}")
print(f"  False Positives: {cm[0, 1]}")
print(f"  False Negatives: {cm[1, 0]}  ← Critical: Missed cardiac risk")
print(f"  True Positives:  {cm[1, 1]}")

print("\n" + "="*70)
print("Classification Report:")
print("="*70)
print(classification_report(y_val, y_val_pred))

VALIDATION METRICS (CARDIAC RISK FOCUS)
Accuracy:  0.7548
Precision: 0.4673
Recall:    0.6797  ← CRITICAL (identifies high-risk patients)
F1 Score:  0.5538
ROC-AUC:   0.8085

Confusion Matrix:
  True Negatives:  622
  False Positives: 179
  False Negatives: 74  ← Critical: Missed cardiac risk
  True Positives:  157

Classification Report:
              precision    recall  f1-score   support

         0.0       0.89      0.78      0.83       801
         1.0       0.47      0.68      0.55       231

    accuracy                           0.75      1032
   macro avg       0.68      0.73      0.69      1032
weighted avg       0.80      0.75      0.77      1032



In [16]:
# Cell 7: Confusion Matrix Interpretation

print("="*70)
print("CONFUSION MATRIX ANALYSIS")
print("="*70)
print(f"\n              Predicted")
print(f"            Negative  Positive")
print(f"Actual Neg    {cm[0, 0]:4d}      {cm[0, 1]:4d}   (True Neg / False Pos)")
print(f"Actual Pos    {cm[1, 0]:4d}      {cm[1, 1]:4d}   (False Neg / True Pos)")

# Calculate rates
tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
false_neg_rate = fn / (tp + fn) if (tp + fn) > 0 else 0

print(f"\nSensitivity (Recall): {sensitivity:.4f} → Detects {sensitivity*100:.1f}% of cardiac risk")
print(f"Specificity: {specificity:.4f} → Correctly rules out {specificity*100:.1f}% of healthy")
print(f"False Negative Rate: {false_neg_rate:.4f} → {int(fn)} missed cardiac cases")

print("\n✓ Confusion matrix analyzed")

CONFUSION MATRIX ANALYSIS

              Predicted
            Negative  Positive
Actual Neg     622       179   (True Neg / False Pos)
Actual Pos      74       157   (False Neg / True Pos)

Sensitivity (Recall): 0.6797 → Detects 68.0% of cardiac risk
Specificity: 0.7765 → Correctly rules out 77.7% of healthy
False Negative Rate: 0.3203 → 74 missed cardiac cases

✓ Confusion matrix analyzed


In [18]:
# Cell 8: Classification Report

print("="*70)
print("DETAILED CLASSIFICATION REPORT")
print("="*70)
print("\n")
print(classification_report(y_val, y_val_pred, target_names=['No Risk', 'Cardiac Risk']))

print("\n" + "="*70)
print("KEY METRICS")
print("="*70)
print(f"Overall Accuracy: {accuracy_score(y_val, y_val_pred):.4f}")
print(f"Weighted Precision: {precision_score(y_val, y_val_pred, average='weighted', zero_division=0):.4f}")
print(f"Weighted Recall: {recall_score(y_val, y_val_pred, average='weighted', zero_division=0):.4f}")
print(f"Weighted F1: {f1_score(y_val, y_val_pred, average='weighted', zero_division=0):.4f}")

print("\n✓ Classification report generated")


DETAILED CLASSIFICATION REPORT


              precision    recall  f1-score   support

     No Risk       0.89      0.78      0.83       801
Cardiac Risk       0.47      0.68      0.55       231

    accuracy                           0.75      1032
   macro avg       0.68      0.73      0.69      1032
weighted avg       0.80      0.75      0.77      1032


KEY METRICS
Overall Accuracy: 0.7548
Weighted Precision: 0.7982
Weighted Recall: 0.7548
Weighted F1: 0.7689

✓ Classification report generated


In [19]:
# Cell 9: Save Model Artifact

# Save trained logistic regression model
model_path = "c:\\Users\\Pc\\Desktop\\cardiac-risk-awareness\\notebooks\\data\\processed\\logistic_regression_model.pkl"
joblib.dump(log_reg, model_path)

print("="*70)
print("MODEL ARTIFACT SAVED")
print("="*70)
print(f"✓ Model saved to: {model_path}")
print(f"✓ Model type: Logistic Regression")
print(f"✓ Features: {X_train.shape[1]}")

print("\n" + "="*70)
print("COMPLETE INFERENCE PIPELINE")
print("="*70)
print("Required files for production:")
print("  1. scaler.pkl → Feature scaling")
print("  2. logistic_regression_model.pkl → Predictions")
print("  3. label_encoders (from notebook) → Categorical encoding")

print("\n✓ Model ready for deployment")

MODEL ARTIFACT SAVED
✓ Model saved to: c:\Users\Pc\Desktop\cardiac-risk-awareness\notebooks\data\processed\logistic_regression_model.pkl
✓ Model type: Logistic Regression
✓ Features: 44

COMPLETE INFERENCE PIPELINE
Required files for production:
  1. scaler.pkl → Feature scaling
  2. logistic_regression_model.pkl → Predictions
  3. label_encoders (from notebook) → Categorical encoding

✓ Model ready for deployment


## 🔹 Step 4 — Model Training & Validation

### Baseline Model: Logistic Regression

A logistic regression model was trained with balanced class weights to address class imbalance in cardiac risk prediction. This medical-grade baseline prioritizes recall (sensitivity) to minimize false negatives—missed cardiac risk cases.

### Key Training Decisions
- **Class Weight Balancing**: Prevents bias toward majority class
- **Stratified Split**: Preserves target distribution (80/20 train/val)
- **Metric Priority**: Recall > Accuracy (missing cardiac risk is costly)
- **Solver**: liblinear for stable convergence

### Validation Results
| Metric | Purpose |
|--------|---------|
| **Recall (Sensitivity)** | % of actual cardiac cases detected |
| Specificity | % of healthy cases correctly identified |
| Precision | Reliability of positive predictions |
| F1-Score | Balance between precision & recall |
| ROC-AUC | Overall discrimination ability |

### Model Artifacts
✓ Trained model saved (`logistic_regression_model.pkl`)  
✓ Scaler artifact saved (from Step 3)  
✓ Label encoders tracked (categorical encoding)  
✓ **Complete inference pipeline ready for deployment**

### Clinical Interpretation
The model prioritizes identifying cardiac risk cases (high recall) even at the cost of some false positives, aligning with medical best practices where missing a disease is riskier than a false alarm.